In [42]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings
warnings.filterwarnings('ignore')
random.seed(1)

In [67]:
def gbdt_lr_train_test(File):
    #GBDT
    start = time.clock()
    train_df, test_df = train_test_split(File, train_size = 0.8,random_state=42)
    X_train = train_df.drop(train_df.columns[0], axis=1)
    y_train = train_df[train_df.columns[0]]
    X_test = test_df.drop(test_df.columns[0], axis=1)
    y_test = test_df[test_df.columns[0]]
    gbclf = GradientBoostingClassifier(n_estimators=20, max_depth=4, verbose=0)
    tuned_parameter = [{'n_estimators':[20,30,40,50], 'max_depth':[5, 6, 7, 8], 'max_features':[0.5]}]
    gs_clf = GridSearchCV(gbclf, tuned_parameter, cv=5, scoring='roc_auc')
    gs_clf.fit(X_train, y_train)
    print('best parameters set found: ')
    print(gs_clf.best_params_)
    
    y_pred_gbdt = gs_clf.predict_proba(X_test)[:, 1]
    gbdt_auc = roc_auc_score(y_test, y_pred_gbdt)
    print('gbdt auc: %.5f' % gbdt_auc)
    
    gbclf=GradientBoostingClassifier(**gs_clf.best_params_, random_state=10)
    gbclf.fit(X_train,y_train)
    X_train_leaves = gbclf.apply(X_train)[:,:,0]
    gbclf.fit(X_test,y_test)
    X_test_leaves = gbclf.apply(X_test)[:,:,0]
    
    #GBDT+LR
    lr = LogisticRegression()
    lr.fit(X_train_leaves, y_train)
    y_pred_gbdtlr1 = lr.predict_proba(X_test_leaves)[:,1]
    gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
    print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)
    
    lr = LogisticRegression(n_jobs=-1)
    X_train_ext = hstack([X_train_leaves, X_train])
    lr.fit(X_train_ext, y_train)
    X_test_ext = hstack([X_test_leaves, X_test])
    y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
    gbdtlr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)
    print('gbdt+lr auc 2: %.5f' % gbdtlr_auc2)
    f_time =time.clock()-start
    print('GBDT+LR time taken: %.2f'% f_time)
    
    #+NB

    start = time.clock()
    gnb= GaussianNB()
    gnb.fit(X_train_leaves, y_train)
    Y_pred_nb=gnb.predict_proba(X_test_leaves)[:,1]
    gnb_auc = roc_auc_score(y_test,Y_pred_nb)
    print('NB auc: %.5f'% gnb_auc)    
    '''    
    gnb.fit(X_train_ext, y_train)
    y_pred_gnb2=gnb.predict_proba(X_test_ext)[:,1]
    gnb_auc2=roc_auc_score(y_test,y_pred_gnb2)
    print('GNB auc2: %.5f' % gnb_auc2)
    '''
    #svc
    svc=SVC(probability=True)
    svc.fit(X_train_leaves, y_train)
    Y_pred_svc=svc.predict_proba(X_test_leaves)[:,1]
    svc_auc=roc_auc_score(y_test,Y_pred_svc)
    print('SVC auc: %.5f' % svc_auc)
    
    svc.fit(X_train_ext, y_train)
    y_pred_svc2=svc.predict_proba(X_test_ext)[:, 1]
    svc_auc2=roc_auc_score(y_test,y_pred_svc2)
    print('SVC auc2: %.5f' % svc_auc2)
    
    #KNN
    knn=KNeighborsClassifier(n_neighbors = 3)
    knn.fit(X_train_leaves, y_train)
    Y_pred_knn=knn.predict_proba(X_test_leaves)[:,1]
    knn_auc=roc_auc_score(y_test,Y_pred_knn)
    print('KNN auc : %.5f' % knn_auc)
    
    knn.fit(X_train_ext, y_train)
    y_pred_knn2=knn.predict_proba(X_test_ext)[:, 1]
    knn_auc2=roc_auc_score(y_test,y_pred_knn2)
    print('KNN auc2: %.5f' % knn_auc2)
    
    #perceptron

    perceptron = Perceptron()
    perceptron.fit(X_train_leaves, y_train)
    y_pred_perc=perceptron.predict(X_test_leaves)
    perc_auc=roc_auc_score(y_test,y_pred_perc)
    print('Perceptron auc : %.5f' % perc_auc)
    
    perceptron.fit(X_train_ext, y_train)
    y_pred_perc2=perceptron.predict(X_test_ext)
    perc_auc2=roc_auc_score(y_test,y_pred_perc2)
    print('Perceptron auc2 : %.5f' % perc_auc2 )
    
    #linear svc

    lin = LinearSVC()
    lin.fit(X_train_leaves, y_train)
    y_pred_lin=lin.predict(X_test_leaves)
    lin_auc=roc_auc_score(y_test,y_pred_lin)
    print('Linear SVC auc : %.5f' % lin_auc)
    
    lin.fit(X_train_ext, y_train)
    y_pred_lin2=lin.predict(X_test_ext)
    lin_auc2=roc_auc_score(y_test,y_pred_lin2)
    print('Linear SVC auc2 : %.5f' % lin_auc2)
    
    #SGD

    sgd = SGDClassifier(loss='log')
    sgd.fit(X_train_leaves, y_train)
    Y_pred_sgd=sgd.predict_proba(X_test_leaves)[:, 1]
    sgd_auc=roc_auc_score(y_test,Y_pred_sgd)
    print('SGD auc : %.5f' % sgd_auc)
    
    sgd.fit(X_train_ext, y_train)
    Y_pred_sgd2=sgd.predict_proba(X_test_ext)[:, 1]
    sgd_auc2=roc_auc_score(y_test,Y_pred_sgd2)
    print('SGD auc2 : %.5f' % sgd_auc2)
    f_time =time.clock()-start
    print('GBDT + other classfier time taken: %.2f'% f_time)    
    
    #XGB
    start = time.clock()
    xgb=XGBClassifier()
    xgb.fit(X_train,y_train)
    Y_pred_xgb=xgb.predict_proba(X_test)[:,1]
    xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
    print('XGB auc : %.5f' % xgb_auc)
    
    #XGB with leaves

    xgb.fit(X_train_leaves,y_train)
    Y_pred_xgb=xgb.predict(X_test_leaves)
    xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
    print('XGB auc with feature transformed: %.5f' % xgb_auc)
    
    f_time =time.clock()-start
    print('XGB time taken: %.2f'% f_time)
    #lightGBM
    start = time.clock()
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    # specify your configurations as a dict
    params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
    }

    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=5)



    y_pred_lgb = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    # eval
    lgb_auc=roc_auc_score(y_test,y_pred_lgb)
    print('lightGBM auc : %.5f' % lgb_auc)
    
    lgb_train=lgb.Dataset(X_train_leaves, y_train)
    lgb_eval=lgb.Dataset(X_test_leaves, y_test, reference=lgb_train)
    gbm = lgb.train(params,
                  lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=5)
    y_pred_lgb2 =gbm.predict(X_test_leaves, num_iteration=gbm.best_iteration)
    lgb_auc2=roc_auc_score(y_test, y_pred_lgb2)
    
    print('lightGBM auc2 : %.5f' % lgb_auc)
    
    print('lightGBM time taken: ', time.clock()-start)

In [45]:
example = pd.read_csv('example.csv')

In [16]:
gbdt_lr_train_test(example) # V3 trial

best parameters set found: 
{'max_depth': 8, 'max_features': 0.5, 'n_estimators': 100}
gbdt auc: 0.94744
gbdt+lr auc 1: 0.47496
gbdt+lr auc 2: 0.47697
GBDT+LR time taken: 40.24
NB auc:  0.728048687278
SVC auc: 0.50000
SVC auc2: 0.50000
KNN auc : 0.79614
KNN auc2: 0.61520
Perceptron auc : 0.51301
Perceptron auc2 : 0.50000
Linear SVC auc : 0.49594
Linear SVC auc2 : 0.52416
SGD auc : 0.56036
SGD auc2 : 0.49957
GBDT + other classfier time taken: 84.77
XGB auc : 0.86043
XGB auc with feature transformed: 0.53599
XGB time taken: 2.75


In [17]:
gbdt_lr_train_test(example) # V3 trial

best parameters set found: 
{'max_depth': 6, 'max_features': 0.5, 'n_estimators': 100}
gbdt auc: 0.94873
gbdt+lr auc 1: 0.48734
gbdt+lr auc 2: 0.81641
GBDT+LR time taken: 32.58
NB auc:  0.843961860358
SVC auc: 0.50000
SVC auc2: 0.50000
KNN auc : 0.85452
KNN auc2: 0.60983
Perceptron auc : 0.61611
Perceptron auc2 : 0.50000
Linear SVC auc : 0.53852
Linear SVC auc2 : 0.50000
SGD auc : 0.66286
SGD auc2 : 0.50000
GBDT + other classfier time taken: 86.49
XGB auc : 0.86043
XGB auc with feature transformed: 0.69166
XGB time taken: 2.68


In [18]:
gbdt_lr_train_test(example) # V3 trial

best parameters set found: 
{'max_depth': 6, 'max_features': 0.5, 'n_estimators': 100}
gbdt auc: 0.94657
gbdt+lr auc 1: 0.48735
gbdt+lr auc 2: 0.81641
GBDT+LR time taken: 32.95
NB auc:  0.843961860358
SVC auc: 0.50000
SVC auc2: 0.50000
KNN auc : 0.85452
KNN auc2: 0.60983
Perceptron auc : 0.61611
Perceptron auc2 : 0.50000
Linear SVC auc : 0.47925
Linear SVC auc2 : 0.48028
SGD auc : 0.62034
SGD auc2 : 0.50000
GBDT + other classfier time taken: 87.54
XGB auc : 0.86043
XGB auc with feature transformed: 0.69166
XGB time taken: 2.60


In [23]:
gbdt_lr_train_test(example) # V3 trial with n_estimator= 20

best parameters set found: 
{'max_depth': 6, 'max_features': 0.5, 'n_estimators': 100}
gbdt auc: 0.94619
gbdt+lr auc 1: 0.48735
gbdt+lr auc 2: 0.81641
GBDT+LR time taken: 32.65
NB auc: 0.84396
SVC auc: 0.50000
SVC auc2: 0.50000
KNN auc : 0.85452
KNN auc2: 0.60983
Perceptron auc : 0.61611
Perceptron auc2 : 0.50000
Linear SVC auc : 0.47830
Linear SVC auc2 : 0.49935
SGD auc : 0.66557
SGD auc2 : 0.50715
GBDT + other classfier time taken: 85.66
XGB auc : 0.93941
XGB auc with feature transformed: 0.69166
XGB time taken: 2.68


In [32]:
gbdt_lr_train_test(example) 

best parameters set found: 
{'max_depth': 6, 'max_features': 0.5, 'n_estimators': 100}
gbdt auc: 0.94810
gbdt+lr auc 1: 0.48738
gbdt+lr auc 2: 0.81641
GBDT+LR time taken: 32.39
NB auc: 0.84396
SVC auc: 0.50000
SVC auc2: 0.50000
KNN auc : 0.85452
KNN auc2: 0.60983
Perceptron auc : 0.61611
Perceptron auc2 : 0.50000
Linear SVC auc : 0.53175
Linear SVC auc2 : 0.55129
SGD auc : 0.58215
SGD auc2 : 0.50000
GBDT + other classfier time taken: 87.13
XGB auc : 0.93941
XGB auc with feature transformed: 0.69166
XGB time taken: 2.69


In [38]:
gbdt_lr_train_test(example) 

best parameters set found: 
{'max_depth': 6, 'max_features': 0.5, 'n_estimators': 100}
gbdt auc: 0.94665
gbdt+lr auc 1: 0.48735
gbdt+lr auc 2: 0.81641
GBDT+LR time taken: 33.04
NB auc: 0.84396
SVC auc: 0.50000
SVC auc2: 0.50000
KNN auc : 0.85452
KNN auc2: 0.60983
Perceptron auc : 0.61611
Perceptron auc2 : 0.50000
Linear SVC auc : 0.52158
Linear SVC auc2 : 0.50000
SGD auc : 0.63871
SGD auc2 : 0.52030
GBDT + other classfier time taken: 88.46
XGB auc : 0.93941
XGB auc with feature transformed: 0.69166
XGB time taken: 2.65


In [46]:
gbdt_lr_train_test(example) 

best parameters set found: 
{'max_depth': 6, 'max_features': 0.5, 'n_estimators': 100}
gbdt auc: 0.94557
gbdt+lr auc 1: 0.48735
gbdt+lr auc 2: 0.81641
GBDT+LR time taken: 32.15
NB auc: 0.84396
SVC auc: 0.50000
SVC auc2: 0.50000
KNN auc : 0.85452
KNN auc2: 0.60983
Perceptron auc : 0.61611
Perceptron auc2 : 0.50000
Linear SVC auc : 0.51555
Linear SVC auc2 : 0.55458
SGD auc : 0.63701
SGD auc2 : 0.45964
GBDT + other classfier time taken: 86.84
XGB auc : 0.93941
XGB auc with feature transformed: 0.69166
XGB time taken: 2.62
[1]	valid_0's l2: 0.235948	valid_0's auc: 0.909065
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.225072	valid_0's auc: 0.923392
[3]	valid_0's l2: 0.213547	valid_0's auc: 0.926876
[4]	valid_0's l2: 0.203327	valid_0's auc: 0.925996
[5]	valid_0's l2: 0.194702	valid_0's auc: 0.928332
[6]	valid_0's l2: 0.187084	valid_0's auc: 0.930558
[7]	valid_0's l2: 0.179158	valid_0's auc: 0.931759
[8]	valid_0's l2: 0.172098	valid_0's auc: 0.93179
[9]	val

In [52]:
gbdt_lr_train_test(example) 

best parameters set found: 
{'max_depth': 6, 'max_features': 0.5, 'n_estimators': 100}
gbdt auc: 0.94738
gbdt+lr auc 1: 0.48735
gbdt+lr auc 2: 0.81641
GBDT+LR time taken: 32.82
NB auc: 0.84396
SVC auc: 0.50000
SVC auc2: 0.50000
KNN auc : 0.85452
KNN auc2: 0.60983
Perceptron auc : 0.61611
Perceptron auc2 : 0.50000
Linear SVC auc : 0.47487
Linear SVC auc2 : 0.50000
SGD auc : 0.56671
SGD auc2 : 0.49914
GBDT + other classfier time taken: 88.83
XGB auc : 0.93941
XGB auc with feature transformed: 0.69166
XGB time taken: 2.74
[1]	valid_0's l2: 0.235948	valid_0's auc: 0.909065
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.225072	valid_0's auc: 0.923392
[3]	valid_0's l2: 0.213547	valid_0's auc: 0.926876
[4]	valid_0's l2: 0.203327	valid_0's auc: 0.925996
[5]	valid_0's l2: 0.194702	valid_0's auc: 0.928332
[6]	valid_0's l2: 0.187084	valid_0's auc: 0.930558
[7]	valid_0's l2: 0.179158	valid_0's auc: 0.931759
[8]	valid_0's l2: 0.172098	valid_0's auc: 0.93179
[9]	val

In [68]:
gbdt_lr_train_test(example) 

best parameters set found: 
{'max_depth': 7, 'max_features': 0.5, 'n_estimators': 50}
gbdt auc: 0.94642
gbdt+lr auc 1: 0.55789
gbdt+lr auc 2: 0.49749
GBDT+LR time taken: 44.90
NB auc: 0.83106
SVC auc: 0.50000
SVC auc2: 0.50000
KNN auc : 0.81779
KNN auc2: 0.60721
Perceptron auc : 0.41106
Perceptron auc2 : 0.50219
Linear SVC auc : 0.54468
Linear SVC auc2 : 0.46483
SGD auc : 0.45510
SGD auc2 : 0.49869
GBDT + other classfier time taken: 55.01
XGB auc : 0.93941
XGB auc with feature transformed: 0.73873
XGB time taken: 1.72
[1]	valid_0's l2: 0.235948	valid_0's auc: 0.909065
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.225072	valid_0's auc: 0.923392
[3]	valid_0's l2: 0.213547	valid_0's auc: 0.926876
[4]	valid_0's l2: 0.203327	valid_0's auc: 0.925996
[5]	valid_0's l2: 0.194702	valid_0's auc: 0.928332
[6]	valid_0's l2: 0.187084	valid_0's auc: 0.930558
[7]	valid_0's l2: 0.179158	valid_0's auc: 0.931759
[8]	valid_0's l2: 0.172098	valid_0's auc: 0.93179
[9]	vali